In [1]:
import os
os.getcwd()

'c:\\Users\\cha\\Desktop\\Research\\챗봇 프로젝트\\25년\\RAG_qdrant_migration\\migration'

In [2]:
from dotenv import load_dotenv
load_dotenv()

# FAISS_VECTORSTORE_PATH = os.getenv("FAISS_VECTORSTORE_PATH")
# FAISS_VECTORSTORE_PATH = "C:\Users\cha\Desktop\Research\챗봇 프로젝트\12월_RAG\ilora-clone\rag_code\vectorstore\abstract_search_store # rag_code\vectorstore\title_search_store"
FAISS_VECTORSTORE_PATH = r"C:\Users\cha\Desktop\Research\챗봇 프로젝트\12월_RAG\ilora-clone\rag_code\vectorstore\title_search_store"
print(FAISS_VECTORSTORE_PATH)

C:\Users\cha\Desktop\Research\챗봇 프로젝트\12월_RAG\ilora-clone\rag_code\vectorstore\title_search_store


In [3]:
os.listdir(FAISS_VECTORSTORE_PATH)

['index.faiss', 'index.pkl']

In [4]:
os.chdir(FAISS_VECTORSTORE_PATH)
os.getcwd()

'C:\\Users\\cha\\Desktop\\Research\\챗봇 프로젝트\\12월_RAG\\ilora-clone\\rag_code\\vectorstore\\title_search_store'

In [5]:
import faiss 
# Faiss 인덱스 파일 로드 
index = faiss.read_index("index.faiss") 
n_vectors = index.ntotal 
dim = index.d 
print("전체 벡터 개수:", n_vectors, "차원:", dim)

전체 벡터 개수: 195739 차원: 1536


In [10]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings  # 혹은 다른 Embeddings
import logging
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


# 2. Qdrant Client 준비
qdrant_client = QdrantClient(url="http://localhost:6333", api_key="abs123!A")
print("Connected to Qdrant.")


C:\Users\cha\AppData\Local\Temp\ipykernel_21768\868732135.py:12: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(url="http://localhost:6333", api_key="abs123!A")


Connected to Qdrant.


In [9]:

collection_name = "pubmed_rag_search"
vector_size = 1536

print(f"Creating/Recreating collection: {collection_name}")
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
)

Creating/Recreating collection: pubmed_rag_search


NameError: name 'qdrant_client' is not defined

In [106]:
# 모두 openai text-embedding-3-small 사용하기로 변경.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 기본적으로 OpenAIEmbeddings 사용
directory = FAISS_VECTORSTORE_PATH
# FAISS 파일 경로 설정
index_file = "index.faiss"
pkl_file = "index.pkl"

# 파일 존재 여부 확인
if not os.path.exists(index_file):
    raise ValueError(f"FAISS index file not found: {index_file}")
if not os.path.exists(pkl_file):
    raise ValueError(f"FAISS pickle file not found: {pkl_file}")

# FAISS 벡터스토어 로드
vectorstore = FAISS.load_local(
    folder_path=".",
    embeddings=embeddings,
    allow_dangerous_deserialization=True  # Pickle 파일 로드 허용
)

In [ ]:
# FAISS 인덱스 로드
faiss_store = FAISS.load_local(".", embeddings, allow_dangerous_deserialization=True)

In [ ]:
# 벡터 차원 확인
vector_dim = faiss_index.d
print(f"벡터 차원: {vector_dim}")

# 벡터 개수 확인
num_vectors = faiss_index.ntotal
print(f"총 벡터 개수: {num_vectors}")

# Qdrant 클라이언트 설정 (로컬 실행)
client = QdrantClient("http://localhost:6333")

# Qdrant 컬렉션 생성 (없으면 생성)
collection_name = "pubmed_rag_search"
vector_size = 1536

print(f"Creating/Recreating collection: {collection_name}")
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
)

print("Qdrant 컬렉션 생성 완료.")


벡터 차원: 1536
총 벡터 개수: 195739
Creating/Recreating collection: pubmed_rag_search


C:\Users\cha\AppData\Local\Temp\ipykernel_1592\1544145871.py:24: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Qdrant 컬렉션 생성 완료.


In [127]:
# Qdrant에 업로드할 포인트 데이터 변환
points = []
for i in tqdm(range(len(faiss_store.index_to_docstore_id)), position=0):
    doc_id = faiss_store.index_to_docstore_id[i]
    doc = faiss_store.docstore.search(doc_id)
    vector = faiss_store.index.reconstruct(i)
    payload = doc.metadata
    point = PointStruct(id=doc_id, vector=vector, payload=payload)
    points.append(point)

100%|██████████| 195739/195739 [04:21<00:00, 749.88it/s]


In [ ]:
# batch size 500
batch_size = 500
for i in tqdm(range(0, len(points), batch_size), position=0):
    batch = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch)


# client.upsert(collection_name=collection_name, points=points)

100%|██████████| 392/392 [07:06<00:00,  1.09s/it]


: 